In [6]:
import numpy as np
from tqdm.auto import tqdm
import pickle
import os

In [ ]:
# Define the base directory (change this if necessary)
base_dir = os.path.dirname(os.path.abspath(''))

# Construct the full path to the pickle file
data_path = os.path.join(base_dir, 'src/api/model')

In [10]:
with open(f'{data_path}/total_chunks.pkl', 'rb') as f:
    total_chunks = pickle.load(f)
with open(f'{data_path}/total_output_clean.pkl', 'rb') as f:
    total_output_clean = pickle.load(f)
with open(f'{data_path}/total_text_embeddings.npy', 'rb') as f:
    total_text_embeddings = np.load(f)
with open(f'{data_path}/total_url_chunks.pkl', 'rb') as f:
    total_url_chunks = pickle.load(f)
with open(f'{data_path}/total_urls.pkl', 'rb') as f:
    total_urls = pickle.load(f)
    
index = 0
print(total_chunks[index])
print("===============")
print(total_output_clean[index])
print("===============")
print(total_url_chunks[index])
print("===============")
print(total_urls[index])
print("===============")

Người Hà Nội 
Bác sĩ, Tiến sĩ Y học
Nhưng tôi ngày càng thấy ba chữ "người Hà Nội" có chút nhạt phai. Người đến sống ở Hà Nội ngày một nhiều, nhưng tinh thần "người Hà Nội" xưa đã biến đổi theo thời gian, là điều những người yêu Hà Nội trăn trở.
Khái niệm "Người Hà Nội" chưa xuất hiện ngay vào thời điểm vua Minh Mạng ra chỉ dụ thành lập tỉnh Hà Nội (1831). Tính cách người Hà Nội kế thừa nét thanh lịch của con người đất kinh kỳ. Thanh lịch là một từ cổ, gồm thanh và lịch. Thanh chỉ sự trong sáng, tự nhiên. Lịch chỉ sự hiểu biết và tuân thủ các quy định, phép tắc.
Nét đặc sắc của người Hà Nội còn do ảnh hưởng của văn hóa phương Tây, nhất là văn hóa Pháp. Qua biến động của thời cuộc, người Hà Nội không quên nỗi đau mất nước nhưng vẫn mở lòng học những cái hay, cái tiến bộ của một nền văn minh mới, làm giàu có thêm cho văn hóa của mình.
Những trái ngọt của tương tác văn minh Đông Tây hầu như diễn ra trên đất Hà Nội: âm nhạc có tân nhạc hay còn gọi là nhạc tiền chiến, văn chương có phong tr

In [11]:
print(len(total_url_chunks))
print(len(total_output_clean))
print(len(total_chunks))
print(len(total_urls))

2355
2355
2355
1759


In [24]:
embeddings = np.load(f'{data_path}/total_text_embeddings.npy')

with open(f'{data_path}/total_url_chunks.pkl', 'rb') as f:
    urls = pickle.load(f)

assert len(embeddings) == len(urls), "Mismatch between embeddings and URLs"

# Mapping embeddings and URLs
mapped_data = [{"embedding": embeddings[i], "url": urls[i]} for i in range(len(embeddings))]

# print("Sample mappings:")
# for i in range(5): 
#     print(f"Embedding {i}: {mapped_data[i]['embedding'][:5]}...") 
#     print(f"URL {i}: {mapped_data[i]['url']}\n")

Sample mappings:
Embedding 0: [-0.01311811 -0.00574908  0.01761944 -0.03840601  0.03109462]...
URL 0: https://vnexpress.net/nguoi-ha-noi-4800840.html


Embedding 1: [ 0.00327813  0.00209516  0.0416055  -0.03610834  0.00491872]...
URL 1: https://vnexpress.net/nguoi-ha-noi-4800840.html


Embedding 2: [ 0.02008127  0.00152695 -0.04942625 -0.04275586 -0.02783655]...
URL 2: https://vnexpress.net/no-thue-va-hoan-xuat-canh-4794407.html


Embedding 3: [ 0.00247487  0.01809504 -0.01125498 -0.0422227  -0.02857456]...
URL 3: https://vnexpress.net/no-thue-va-hoan-xuat-canh-4794407.html


Embedding 4: [ 0.04277795  0.00149078 -0.02129374 -0.01241409  0.00468695]...
URL 4: https://vnexpress.net/labubu-va-pikalong-4795554.html




In [21]:
# Constants
PINECONE_API_KEY = "45f89d4b-e2d1-4a98-984a-0f5c4bf637a7"
PINECONE_INDEX_NAME = "vnexpress-10-10-2024"
PINECONE_DIMENSION = 1024
PINECONE_METRIC = "cosine"
PINECONE_ENVIRONMENT = "aws"
PINECONE_REGION = "us-east-1"
BATCH_SIZE = 100

In [25]:
import os
from pinecone import Pinecone, ServerlessSpec
from tqdm import tqdm
import time

# Initialize Pinecone
pc = Pinecone(api_key=PINECONE_API_KEY)

# List existing indexes
existing_indexes = pc.list_indexes().names()
print("Existing indexes:", existing_indexes)

# Create index if it doesn't exist
if PINECONE_INDEX_NAME not in existing_indexes:
    pc.create_index(
        name=PINECONE_INDEX_NAME, 
        dimension=PINECONE_DIMENSION, 
        metric=PINECONE_METRIC,
        spec=ServerlessSpec(
            cloud=PINECONE_ENVIRONMENT,
            region=PINECONE_REGION
        )
    )
    print(f"Index '{PINECONE_INDEX_NAME}' created.")
    time.sleep(5)  # Wait for the index to be fully created

# Access the index
index = pc.Index(PINECONE_INDEX_NAME)

# Upsert data
for i in tqdm(range(0, len(mapped_data), BATCH_SIZE)):
    batch_data = mapped_data[i:i + BATCH_SIZE]
    
    to_upsert = [(str(i + j), batch_data[j]["embedding"], {"url": batch_data[j]["url"]}) 
                 for j in range(len(batch_data))]
    
    # Upsert the batch to Pinecone
    index.upsert(vectors=to_upsert)

print(f"Successfully uploaded {len(to_upsert)} vectors to the Pinecone index.")


Existing indexes: ['vnexpress-10-10-2024']


100%|██████████| 24/24 [03:29<00:00,  8.75s/it]

Successfully uploaded 55 vectors to the Pinecone index.


In [5]:
# import pickle

# with open('/kaggle/input/vn-news/total_url_chunks.pkl', 'rb') as file:
#     total_url_chunks = pickle.load(file)

# with open('/kaggle/input/vn-news-output/total_output_clean.pkl', 'rb') as file:
#     total_output_clean = pickle.load(file)

# mapped_data = list(zip(total_url_chunks, total_output_clean))

# for url_chunk, output_clean in mapped_data[:5]:
#     print(f"URL Chunk: {url_chunk}")
#     print(f"Content: {output_clean}\n")
    
# mapped_data = {key: (total_url_chunks[key], total_output_clean[key]) for key in total_url_chunks if key in total_output_clean}

# for key, (url_chunk, output_clean) in list(mapped_data.items())[:5]:
#     print(f"Key: {key}")
#     print(f"URL Chunk: {url_chunk}")
#     print(f"Content: {output_clean}\n")


URL Chunk: https://vnexpress.net/xu-huong-tuyen-dung-ceo-part-time-4797751.html

Content: Why do you often wake up in the middle of the night: diet, nocturia, age, stress anxiety are all factors that cause many people to wake up in the middle of the night without being able to fall asleep again. Have you ever stared at the empty space at 3: 00 or 4: 00 a. m.? Is that just bad habit or is something more serious going on and why does it always happen at this time? "We start getting shallow sleep around 4 to 5: 00 p. m.," says Lisa Artis, deputy executive director of the charity The Sleep Charity. When people are at this stage, it's easy to wake up. If you normally go to bed around 11: 00 p. m., getting up at 4: 00 a. m. is likely. There are many factors that lead to these unwanted awakenings. "Sleep is regulated by the body's circadian clock, or circadian rhythm. One of the most important circadian rhythms is the sleep-wake cycle," says Artis. Sleep is regulated by two hormones, melatoni